In [ ]:
import os
from zod import ZodFrames
from zod.constants import AnnotationProject
from typing import List
import cv2  # Assuming you have OpenCV installed for image dimension extraction

# Initialize the dataset
dataset_root = "/Users/varunravi/Desktop/OD"  # Your local path to ZOD
version = "full"  # Choose "mini" or "full"
zod_frames = ZodFrames(dataset_root=dataset_root, version=version)

### Creating Label files in YOLO format

In [ ]:
import zod.constants as constants
from zod import ObjectAnnotation
# Define paths for images and labels

labels_dir = os.path.join(dataset_root, "dataset_zod/labels_new")

# Ensure labels directory exists
os.makedirs(labels_dir, exist_ok=True)

# Helper function to convert Box2D annotations to YOLO format
def convert_to_yolo_format(box, img_width, img_height):
    x, y, w, h = box
    center_x = x + (w / 2)
    center_y = y + (h / 2)

    center_x_normalized = center_x / img_width
    center_y_normalized = center_y / img_height
    width_normalized = w / img_width
    height_normalized = h / img_height
    return center_x_normalized, center_y_normalized, width_normalized, height_normalized

# Dictionary to map annotation names to class IDs
class_mapping = {}
next_class_id = 0

# Set the maximum number of frames to process
max_frames = 10000
processed_frames = 0

# Process each frame in the training set
for frame_id in zod_frames.get_split(constants.TRAIN):
    if processed_frames >= max_frames:
        break  # Stop after processing 10,000 frames

    zod_frame = zod_frames[frame_id]
    #print(frame_id)
    annotations: List[ObjectAnnotation] = zod_frame.get_annotation(AnnotationProject.OBJECT_DETECTION)
    

    # Define path to the specific frame's image folder
    images_dir = os.path.join(dataset_root, f"single_frames/{frame_id}/camera_front_blur")

    # Fetch the image file from the camera_front_blur folder
    image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

    if not image_files:
        print(f"No images found in the directory: {images_dir}")
        continue

    # Assume only one image file per folder; select the first one
    image_file = image_files[0]
    image_path = os.path.join(images_dir, image_file)

    # Load image to get its dimensions
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image at path: {image_path}")
        continue
    img_height, img_width = image.shape[:2]

    # Prepare the label file name, corresponding to the image name without extension
    label_file_name = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(labels_dir, label_file_name)
    
    with open(label_path, "w") as label_file:
        for annotation in annotations:
            # Extract the 2D bounding box if it exists
            box2d = annotation.box2d
            if box2d:
                # Convert annotation box2d to YOLO format
                center_x, center_y, width, height = convert_to_yolo_format(box2d.xywh, img_width, img_height)

                # Get the class name for the annotation
                annotation_name = annotation.name

                # Check if the annotation name already has a class ID
                if annotation_name not in class_mapping:
                    # Assign a new class ID to this annotation name
                    class_mapping[annotation_name] = next_class_id
                    next_class_id += 1

                # Retrieve the class ID for this annotation
                class_id = class_mapping[annotation_name]

                # Write to file: class_id + yolo formatted bounding box
                label_file.write(f"{class_id} {center_x} {center_y} {width} {height}\n")

    processed_frames += 1  # Increment the count of processed frames

# Print the class mapping for reference
print("Class mapping:", class_mapping)

In [ ]:
import os
import shutil

# Define your folder paths
labels_folder = '/Users/varunravi/Desktop/OD/dataset_zod/labels_new'
images_folder = '/Users/varunravi/Desktop/OD/zod_image_labels/images'
images_new_folder = '/Users/varunravi/Desktop/OD/dataset_zod/images_new'

# Create the new folder if it doesn't exist
os.makedirs(images_new_folder, exist_ok=True)

# Get list of filenames (without extension) in the labels folder
label_files = {os.path.splitext(f)[0] for f in os.listdir(labels_folder) if f.endswith('.txt')}

# Iterate through images and move the corresponding ones
for image_file in os.listdir(images_folder):
    image_name, image_ext = os.path.splitext(image_file)

    # Check if the image name matches any label filename
    if image_name in label_files:
        source_path = os.path.join(images_folder, image_file)
        destination_path = os.path.join(images_new_folder, image_file)
        
        # Move the image to the new folder
        shutil.move(source_path, destination_path)

print(f"Images matching the labels have been moved to '{images_new_folder}'.")

### Visualize Bounding Boxes

In [ ]:
import os
import cv2

# Define the folder paths
images_folder = '/Users/varunravi/Desktop/OD/dataset_zod/images_new'
labels_folder = '/Users/varunravi/Desktop/OD/dataset_zod/labels_new'
annotated_folder = '/Users/varunravi/Desktop/OD/dataset_zod/annotated_frames'

# Create the annotated folder if it doesn't exist
os.makedirs(annotated_folder, exist_ok=True)

# Define class names and corresponding colors
class_mappings = {'TrafficSign': 0, 'PoleObject': 1, 'TrafficSignal': 2, 'Vehicle': 3, 'TrafficGuide': 4, 'VulnerableVehicle': 5, 'Pedestrian': 6, 'DynamicBarrier': 7, 'TrafficBeacon': 8, 'Unclear': 9, 'Animal': 10}

class_names = {v: k for k, v in class_mappings.items()}

# Define colors for each class
class_colors = {
    0: (0, 255, 255),   # Yellow for TrafficSign
    1: (128, 128, 128), # Gray for PoleObject
    2: (255, 0, 0),     # Blue for Vehicle
    3: (0, 0, 255),     # Red for VulnerableVehicle
    4: (0, 255, 0),     # Green for TrafficSignal
    5: (255, 255, 0),   # Cyan for TrafficGuide
    6: (255, 0, 255),   # Magenta for Pedestrian
    7: (0, 165, 255),   # Orange for Animal
    8: (255, 255, 255), # White for Unclear
    9: (128, 0, 128),   # Purple for TrafficBeacon
    10: (0, 128, 255),  # Light Orange for DynamicBarrier
}

# Function to convert Box2D.xywh to YOLO format
def convert_to_yolo_format(box, img_width, img_height):
    x, y, w, h = box
    center_x = (x + w / 2) / img_width
    center_y = (y + h / 2) / img_height
    width = w / img_width
    height = h / img_height
    return center_x, center_y, width, height

# Function to draw bounding boxes on images
def annotate_image(image_path, label_path, save_path):
    # Read the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    # Read the label file
    with open(label_path, 'r') as file:
        for line in file:
            # Parse the label line in Box2D.xywh format
            class_id, x, y, box_width, box_height = map(float, line.split())
            class_id = int(class_id)
            
            # Convert Box2D.xywh to YOLO format
            x_center, y_center, box_width, box_height = convert_to_yolo_format((x, y, box_width, box_height), width, height)

            # Ensure values are within bounds [0, 1]
            x_center = min(max(x_center, 0), 1)
            y_center = min(max(y_center, 0), 1)
            box_width = min(max(box_width, 0), 1)
            box_height = min(max(box_height, 0), 1)
            
            # Convert YOLO format to pixel values for drawing
            x_center *= width
            y_center *= height
            box_width *= width
            box_height *= height
            
            # Calculate top-left corner and bottom-right corner of the bounding box
            x1 = int(x_center - box_width / 2)
            y1 = int(y_center - box_height / 2)
            x2 = int(x_center + box_width / 2)
            y2 = int(y_center + box_height / 2)
            
            # Choose color and class name
            color = class_colors.get(class_id, (255, 255, 255))  # Default to white if not specified
            class_name = class_names.get(class_id, 'Unknown')
            
            # Draw the bounding box on the image
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            # Put class name and class ID as text above the bounding box
            cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Save the annotated image
    cv2.imwrite(save_path, image)

# Process each image in the images folder
for image_file in os.listdir(images_folder):
    image_name, image_ext = os.path.splitext(image_file)
    image_path = os.path.join(images_folder, image_file)
    label_path = os.path.join(labels_folder, f"{image_name}.txt")
    save_path = os.path.join(annotated_folder, image_file)
    
    # Check if a corresponding label file exists
    if os.path.exists(label_path):
        # Annotate and save the image
        annotate_image(image_path, label_path, save_path)

print(f"Annotated images have been saved in '{annotated_folder}'.")

In [ ]:
import os
import shutil
import random

# Define the folder paths
dataset_folder = '/Users/varunravi/Desktop/OD/dataset_zod'
images_folder = os.path.join(dataset_folder, 'images_new')
labels_folder = os.path.join(dataset_folder, 'labels_new')

# Define the paths for train, val, and test folders
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(os.path.join(dataset_folder, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(dataset_folder, split, 'labels'), exist_ok=True)

# Get the list of images and ensure corresponding labels exist
image_files = [f for f in os.listdir(images_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(image_files)  # Shuffle the files for random splitting

# Define split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Calculate the number of files for each split
total_images = len(image_files)
train_count = int(total_images * train_ratio)
val_count = int(total_images * val_ratio)

# Split the files into train, val, and test sets
train_files = image_files[:train_count]
val_files = image_files[train_count:train_count + val_count]
test_files = image_files[train_count + val_count:]

# Function to move files to respective directories
def move_files(file_list, split):
    for file_name in file_list:
        base_name, ext = os.path.splitext(file_name)
        image_path = os.path.join(images_folder, file_name)
        label_path = os.path.join(labels_folder, f"{base_name}.txt")
        
        # Check if the label file exists for the image
        if os.path.exists(label_path):
            # Define destination paths
            dest_image_path = os.path.join(dataset_folder, split, 'images', file_name)
            dest_label_path = os.path.join(dataset_folder, split, 'labels', f"{base_name}.txt")
            
            # Move image and label to respective split folders
            shutil.copy(image_path, dest_image_path)
            shutil.copy(label_path, dest_label_path)

# Move files into train, val, and test folders
move_files(train_files, 'train')
move_files(val_files, 'val')
move_files(test_files, 'test')

print(f"Dataset split into train, val, and test folders under '{dataset_folder}'.")

### Creating data.yaml file

In [ ]:
import os

# Define the dataset folder path
dataset_folder = '/Users/varunravi/Desktop/OD/dataset_zod'

# Define the paths for train, val, and test sets
train_path = os.path.join(dataset_folder, 'train', 'images')
val_path = os.path.join(dataset_folder, 'valid', 'images')
test_path = os.path.join(dataset_folder, 'test', 'images')

# Define the class mappings
class_names = [
    'TrafficSign', 'PoleObject', 'Vehicle', 'VulnerableVehicle', 
    'TrafficSignal', 'TrafficGuide', 'Pedestrian', 'Animal', 
    'Unclear', 'TrafficBeacon', 'DynamicBarrier'
]

# Create the data.yaml content
data_yaml_content = f"""
# YOLO data configuration file

path: {dataset_folder}  # Root path to your dataset
train: {train_path}  # Train images directory
val: {val_path}  # Validation images directory
test: {test_path}  # Test images directory

# Number of classes
nc: {len(class_names)}

# Class names
names: {class_names}
"""

# Save the data.yaml file
yaml_path = os.path.join(dataset_folder, 'data.yaml')
with open(yaml_path, 'w') as file:
    file.write(data_yaml_content.strip())

print(f"'data.yaml' file has been created at: {yaml_path}")

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.yaml")

In [ ]:
results = model.train(data="/Users/varunravi/Desktop/OD/dataset_zod/data.yaml", epochs=50, conf = 0.5, iou=0.45)

In [ ]:
import os
import cv2
from tqdm import tqdm
from ultralytics import YOLO
import random

# Load the pretrained YOLOv8 model
model = YOLO("/opt/homebrew/runs/detect/train29/weights/best.pt")

# Define the path to the test images and the path to save the results
test_images_folder = "/Users/varunravi/Desktop/OD/dataset_zod/test/images"
pred_folder = "/Users/varunravi/Desktop/OD/dataset_zod/pred"

# Create the 'pred' folder if it doesn't exist
os.makedirs(pred_folder, exist_ok=True)

# Get list of test images
test_images = [os.path.join(test_images_folder, f) for f in os.listdir(test_images_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Generate random colors for each class
class_colors = {name: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for name in model.names.values()}

def draw_sleek_bounding_boxes(image, boxes, labels, scores, class_colors):
    """Draws sleek and elegant bounding boxes with different colors for each class."""
    for box, label, score in zip(boxes, labels, scores):
        # Extract box coordinates
        x_min, y_min, x_max, y_max = map(int, box)

        # Use class-specific color
        color = class_colors[label]

        # Draw the rectangle with a sleek thin line
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2, lineType=cv2.LINE_AA)

        # Prepare the label with the score
        label_text = f"{label}: {score:.2f}"
        (label_width, label_height), baseline = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        
        # Draw a semi-transparent background for the label
        overlay = image.copy()
        label_bg_top_left = (x_min, y_min - label_height - baseline - 10)
        label_bg_bottom_right = (x_min + label_width + 10, y_min)
        cv2.rectangle(overlay, label_bg_top_left, label_bg_bottom_right, color, -1)
        alpha = 0.6
        cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)

        # Draw the label text
        label_pos = (x_min + 5, y_min - baseline - 5)
        cv2.putText(image, label_text, label_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, lineType=cv2.LINE_AA)

    return image

# Run batched inference on the list of images with progress bar
for image_path in tqdm(test_images, desc="Processing images"):
    # Run inference on the current image
    result = model(image_path)[0]  # Get the first (and only) result

    # Extract the original filename
    original_filename = os.path.basename(image_path)

    # Get bounding boxes, labels, and scores from the result
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding box coordinates
    scores = result.boxes.conf.cpu().numpy()  # Confidence scores
    labels = [model.names[int(cls)] for cls in result.boxes.cls.cpu().numpy()]  # Class names

    # Load the original image
    image = cv2.imread(image_path)

    # Draw sleek and elegant bounding boxes with class-specific colors
    sleek_image = draw_sleek_bounding_boxes(image, boxes, labels, scores, class_colors)

    # Define the path to save the result image in the 'pred' folder
    save_path = os.path.join(pred_folder, original_filename)

    # Save the result image
    cv2.imwrite(save_path, sleek_image)  # Save to disk with the same filename in the 'pred' folder

    # Display the result image
    cv2.imshow("Detected Image", sleek_image)
    
    # Wait for a key press to close the image window
    cv2.waitKey(1)  # Press any key to advance to the next image

# Close all OpenCV windows after processing
cv2.destroyAllWindows()

print(f"Processed images saved in: {pred_folder}")

In [1]:
import os
import cv2
from tqdm import tqdm
from ultralytics import YOLO
import random

# Load the pretrained YOLOv8 model
model = YOLO("/opt/homebrew/runs/detect/train29/weights/best.pt")

# Define the path to the sequence images and the path to save the results video
sequence_folder = "/Users/varunravi/Desktop/OD/sequences/000002/camera_front_blur"
video_save_path = "/Users/varunravi/Desktop/OD/sequences/000002/detected_sequence.mp4"

# Get list of images in the given order (without sorting)
sequence_images = [
    os.path.join(sequence_folder, f) for f in os.listdir(sequence_folder) if f.endswith(('.jpg', '.jpeg', '.png'))
]

# Check if there are images to process
if not sequence_images:
    print("No images found in the sequence folder.")
    exit()

# Read the first image to determine the video dimensions
first_image = cv2.imread(sequence_images[0])
if first_image is None:
    print("Failed to load the first image.")
    exit()

# Get image dimensions (width, height)
height, width, _ = first_image.shape

# Define the video writer with codec, output path, fps, and frame size
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 format
fps = 7  # Frames per second for the output video
video_writer = cv2.VideoWriter(video_save_path, fourcc, fps, (width, height))

# Generate random colors for each class
class_colors = {name: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for name in model.names.values()}

def draw_sleek_bounding_boxes(image, boxes, labels, scores, class_colors):
    """Draws sleek and elegant bounding boxes with different colors for each class."""
    for box, label, score in zip(boxes, labels, scores):
        # Extract box coordinates
        x_min, y_min, x_max, y_max = map(int, box)

        # Use class-specific color
        color = class_colors[label]

        # Draw the rectangle with a sleek thin line
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2, lineType=cv2.LINE_AA)

        # Prepare the label with the score
        label_text = f"{label}: {score:.2f}"
        (label_width, label_height), baseline = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

        # Draw a semi-transparent background for the label
        overlay = image.copy()
        label_bg_top_left = (x_min, y_min - label_height - baseline - 10)
        label_bg_bottom_right = (x_min + label_width + 10, y_min)
        cv2.rectangle(overlay, label_bg_top_left, label_bg_bottom_right, color, -1)
        alpha = 0.6
        cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)

        # Draw the label text
        label_pos = (x_min + 5, y_min - baseline - 5)
        cv2.putText(image, label_text, label_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, lineType=cv2.LINE_AA)

    return image

# Process each image, run inference, draw bounding boxes, and write to video
for image_path in tqdm(sequence_images, desc="Processing images"):
    # Run inference on the current image
    result = model(image_path)[0]  # Get the first (and only) result

    # Get bounding boxes, labels, and scores from the result
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding box coordinates
    scores = result.boxes.conf.cpu().numpy()  # Confidence scores
    labels = [model.names[int(cls)] for cls in result.boxes.cls.cpu().numpy()]  # Class names

    # Load the original image
    image = cv2.imread(image_path)

    # Draw sleek and elegant bounding boxes with class-specific colors
    sleek_image = draw_sleek_bounding_boxes(image, boxes, labels, scores, class_colors)

    # Write the result image to the video
    video_writer.write(sleek_image)

# Release the video writer
video_writer.release()

# Inform the user about the saved video
print(f"Video successfully saved at: {video_save_path}")

# Optionally display the video
cap = cv2.VideoCapture(video_save_path)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow('Detected Video', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):  # Press 'q' to exit the video display
        break

cap.release()
cv2.destroyAllWindows()

Processing images:   0%|          | 0/201 [00:00<?, ?it/s]


image 1/1 /Users/varunravi/Desktop/OD/sequences/000002/camera_front_blur/000002_romeo_2022-06-13T10:50:07.785457Z.jpg: 384x640 14 Vehicles, 24.1ms
Speed: 1.3ms preprocess, 24.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


Processing images:   0%|          | 1/201 [00:00<02:49,  1.18it/s]


image 1/1 /Users/varunravi/Desktop/OD/sequences/000002/camera_front_blur/000002_romeo_2022-06-13T10:50:16.499809Z.jpg: 384x640 1 TrafficSign, 4 Vehicles, 27.5ms
Speed: 1.3ms preprocess, 27.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)


Processing images:   1%|          | 2/201 [00:00<01:25,  2.34it/s]

...


image 1/1 /Users/varunravi/Desktop/OD/sequences/000002/camera_front_blur/000002_romeo_2022-06-13T10:50:17.192997Z.jpg: 384x640 1 TrafficSign, 5 Vehicles, 24.5ms
Speed: 1.7ms preprocess, 24.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)


Processing images: 100%|█████████▉| 200/201 [00:30<00:00,  7.28it/s]


image 1/1 /Users/varunravi/Desktop/OD/sequences/000002/camera_front_blur/000002_romeo_2022-06-13T10:50:14.222195Z.jpg: 384x640 6 Vehicles, 24.4ms
Speed: 1.4ms preprocess, 24.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)


Processing images: 100%|██████████| 201/201 [00:30<00:00,  6.53it/s]


Video successfully saved at: /Users/varunravi/Desktop/OD/sequences/000002/detected_sequence.mp4


In [ ]:
imageio.mimsave(gif_save_path, gif_frames, duration=0.5)  # Duration between frames (in seconds)

In [1]:
import os
import cv2
from tqdm import tqdm

# Define the path to the folder containing images and the output video file
image_folder = "/Users/varunravi/Desktop/OD/sequences/000002/camera_front_blur"  # Replace with your folder path
video_save_path = "/Users/varunravi/Desktop/OD/sequences/output_video.mp4"  # Replace with your desired output video path

# Get list of images sorted in the correct order
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
image_files.sort()  # Sort files to ensure correct order

# Check if there are images to process
if not image_files:
    print("No images found in the folder.")
    exit()

# Read the first image to determine the video dimensions
first_image_path = os.path.join(image_folder, image_files[0])
first_image = cv2.imread(first_image_path)

if first_image is None:
    print(f"Failed to load the first image: {first_image_path}")
    exit()

# Get image dimensions (width, height)
height, width, _ = first_image.shape

# Define the video writer with codec, output path, fps, and frame size
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 format
fps = 30  # Frames per second for the output video
video_writer = cv2.VideoWriter(video_save_path, fourcc, fps, (width, height))

# Write each image to the video
for image_file in tqdm(image_files, desc="Creating video"):
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Failed to load image: {image_path}")
        continue

    # Write the image frame to the video
    video_writer.write(image)

# Release the video writer
video_writer.release()

# Inform the user about the saved video
print(f"Video successfully saved at: {video_save_path}")

Creating video: 100%|██████████| 201/201 [00:10<00:00, 19.97it/s]

Video successfully saved at: /Users/varunravi/Desktop/OD/sequences/output_video.mp4


In [11]:
import os
from ultralytics import YOLO
import cv2
import random

# Define paths
VIDEOS_DIR = '/Users/varunravi/Desktop/OD/sequences'
video_path = '/Users/varunravi/Desktop/OD/sequences/output_video.mp4'
video_path_out = '/Users/varunravi/Desktop/OD/sequences/output_video_od.mp4'

# Load video
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    print("Failed to read the video.")
    cap.release()
    exit()

# Get frame dimensions and set up video writer
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'mp4v'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

# Load the custom model
model_path = os.path.join('.', 'runs', 'detect', 'train29', 'weights', 'best.pt')
model = YOLO("/opt/homebrew/runs/detect/train29/weights/best.pt")  # Load your custom YOLO model

# Set the detection threshold
threshold = 0.5

# Generate random colors for each class
class_colors = {name: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for name in model.names.values()}

while ret:
    # Perform detection on the current frame
    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            # Get the class name and color
            class_name = results.names[int(class_id)]
            color = class_colors[class_name]

            # Draw the bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)

            # Draw the label text
            cv2.putText(frame, class_name, (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    # Write the processed frame to the output video
    out.write(frame)

    # Read the next frame
    ret, frame = cap.read()

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved at: {video_path_out}")


0: 384x640 6 Vehicles, 25.3ms
Speed: 2.1ms preprocess, 25.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Vehicles, 23.9ms
Speed: 1.9ms preprocess, 23.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 Vehicles, 21.9ms
Speed: 1.5ms preprocess, 21.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TrafficSign, 7 Vehicles, 21.9ms
Speed: 1.3ms preprocess, 21.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 Vehicles, 23.0ms
Speed: 1.3ms preprocess, 23.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 Vehicles, 21.2ms
Speed: 1.3ms preprocess, 21.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 Vehicles, 21.9ms
Speed: 1.3ms preprocess, 21.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 Vehicles, 2 TrafficGuides, 22.8ms
Speed: 1.4ms preprocess, 22.8ms inference

### Segmentation

In [12]:
from collections import defaultdict

import cv2

from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

track_history = defaultdict(lambda: [])

model = YOLO("yolov8n-seg.pt")  # segmentation model
cap = cv2.VideoCapture("/Users/varunravi/Desktop/OD/sequences/output_video.mp4")
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

out = cv2.VideoWriter("instance-segmentation-object-tracking.avi", cv2.VideoWriter_fourcc(*"MJPG"), fps, (w, h))

while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    annotator = Annotator(im0, line_width=2)

    results = model.track(im0, persist=True)

    if results[0].boxes.id is not None and results[0].masks is not None:
        masks = results[0].masks.xy
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for mask, track_id in zip(masks, track_ids):
            color = colors(int(track_id), True)
            txt_color = annotator.get_txt_color(color)
            annotator.seg_bbox(mask=mask, mask_color=color, label=str(track_id), txt_color=txt_color)

    out.write(im0)
    cv2.imshow("instance-segmentation-object-tracking", im0)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

out.release()
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 car, 1 bus, 58.7ms
Speed: 2.1ms preprocess, 58.7ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 43.2ms
Speed: 1.4ms preprocess, 43.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 buss, 37.4ms
Speed: 1.4ms preprocess, 37.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 truck, 31.0ms
Speed: 1.3ms preprocess, 31.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 34.2ms
Speed: 1.3ms preprocess, 34.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 2 trucks, 30.4ms
Speed: 1.2ms preprocess, 30.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 trucks, 30.4ms
Speed: 1.2ms preprocess, 30.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 trucks, 30.4ms
Speed: 1.2ms preprocess, 30.4ms inference, 1.2ms postproce

: 